# Classify NDWI image using defined thresholds

### !! This notebook is meant to be used for a large area.
### !! STEP 1, 2 and 3 might have to be run one at a time.
### !! Each step will load input data and save output to file.

## Load packages

In [1]:
import os
import xarray as xr
import numpy as np
import pandas as pd
import geopandas as gpd

import sys
sys.path.append('../Scripts')
from deafrica_spatialtools import xr_rasterize

from datacube.utils.cog import write_cog

In [2]:
# define area name

area_name = 'Sahel'
if not os.path.exists(area_name): os.mkdir(area_name)

## Convert to one mosaic (only do it once)

In [3]:
# make tif

if not os.path.exists(f"NDWI_composite/{area_name.lower()}_NDWI_mosaic.vrt"):
    os.chdir('NDWI_composite')
    os.system(f"gdalbuildvrt {area_name.lower()}_NDWI_mosaic.vrt {area_name.lower()}_NDWI_tile*.tif")
    #os.system("gdal_translate "\
    #   "-co BIGTIFF=YES "\
    #   "-co COMPRESS=DEFLATE "\
    #   "-co ZLEVEL=9 "\
    #   "-co PREDICTOR=1 "\
    #   "-co TILED=YES "\
    #   "-co BLOCKXSIZE=1024 "\
    #   "-co BLOCKYSIZE=1024 "\
    #   +f"{area_name.lower()}_NDWI_mosaic.vrt "+ f"{area_name.lower()}_NDWI_mosaic.tif")
    os.chdir('../')

## STEP 1 - Load NDWI mosaic and classify into bins of different water detection frequencies

In [ ]:
if os.path.exists(f"NDWI_composite/{area_name.lower()}_NDWI_mosaic.tif"):
    ds = xr.open_rasterio(f"NDWI_composite/{area_name.lower()}_NDWI_mosaic.tif", chunks={'x':2000,'y':2000}).squeeze()
else:
    ds = xr.open_rasterio(f"NDWI_composite/{area_name.lower()}_NDWI_mosaic.vrt", chunks={'x':2000,'y':2000}).squeeze()

In [ ]:
dataset = ds.to_dataset(name='ndwi')

In [ ]:
# fix thresholds
thresh = [-0.07, -0.05, -0.03, 0.03]

n_class = len(thresh)+1
frac_sample = [0.1, 0.1, 0.2, 0.3, 0.3]

In [ ]:
# classify

dataset['label'] = (dataset.ndwi*0).astype(np.uint8)

dataset['label'] +=(dataset.ndwi<thresh[0]).astype(np.uint8)*1

for i in range(2, n_class):
    dataset['label'] += ((dataset.ndwi>=thresh[i-2]) & (dataset.ndwi<thresh[i-1])).astype(np.uint8)*i

dataset['label'] += (dataset.ndwi>=thresh[-1]).astype(np.uint8)*n_class

# ndwi=0 is not valid
dataset['label'] = dataset.label.where(dataset.ndwi!=0, 0)

dataset['label'].attrs = dataset.ndwi.attrs

In [ ]:
%%time 

write_cog(dataset.label.compute(), f'{area_name}/{area_name}_label_unmasked.tif')

## STEP 2 - Clip to AEZ (excluding large_water_bodies)

In [4]:
ds = xr.open_rasterio(f'{area_name}/{area_name}_label_unmasked.tif').squeeze()
dataset = ds.to_dataset(name='label')

In [5]:
#load shapefile
gdf = gpd.read_file(f'shapes/AEZs_ExcludeLargeWB/AEZs_ExcludeLargeWB_update_{area_name}.shp')

#rasterize shapeile
mask = xr_rasterize(gdf=gdf,
                     da=dataset.label, export_tiff= f'{area_name}/{area_name}_mask.tif')

Rasterizing to match xarray.DataArray dimensions (118904, 221753) and projection system/CRS (e.g. +proj=cea +lat_ts=30 +lon_0=0 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs=True)
Exporting GeoTIFF to Sahel/Sahel_mask.tif


/env/lib/python3.6/site-packages/datacube/helpers.py:34: DeprecationWarning: Function datacube.helpers.write_geotiff is deprecated,
please use datacube.utils.cog.write_cog instead
  category=DeprecationWarning)


## STEP 3 - Make the classified NDWI image

In [6]:
ds = xr.open_rasterio(f'{area_name}/{area_name}_label_unmasked.tif').squeeze()
dataset = ds.to_dataset(name='label')

In [7]:
mask =  xr.open_rasterio(f'{area_name}/{area_name}_mask.tif').squeeze()

In [8]:
dataset['label'] = dataset.label.where(mask, 0)

In [9]:
%%time

# save classes

write_cog(dataset.label.compute(), f'{area_name}/{area_name}_label.tif')

CPU times: user 5min 22s, sys: 24.9 s, total: 5min 47s
Wall time: 5min 47s


PosixPath('Sahel/Sahel_label.tif')